# 计算细胞周期特征

本笔记本用于计算每个细胞类型的细胞周期状态特征。主要功能包括：

- 从 pairs.gz 文件中提取细胞周期相关特征
- 计算 near%、mitotic%、farAvg 和 repli score 等指标
- 将结果保存到 HiRES_emb_metadata.xlsx 文件中
- 使用并发处理加速计算过程

这些特征用于后续的细胞周期状态分配和分析。

In [1]:
# 本文件用于计算每个细胞类型的细胞周期状态特征
# 特征包括：near%、mitotic%、farAvg、repli score，结果保存至 HiRES_emb_metadata.xlsx

# 导入系统模块，便于后续添加自定义路径
# import sys
# import os

# # 添加 CHARMtools 工具包的路径
# # 获取当前工作目录并正确设置路径
# current_dir = os.getcwd()
# print("当前工作目录:", current_dir)

# # 添加CHARMtools_snapshot的绝对路径
# charmtools_path = os.path.join(current_dir, "HiRES", "analysis_and_plot_notebooks", "CHARMtools")
# sys.path.insert(0, charmtools_path)

# # 也添加相对路径作为备选
# sys.path.append("../CHARMtools/")

# print("添加的绝对路径:", charmtools_path)
# print("路径是否存在:", os.path.exists(charmtools_path))
# print("当前Python搜索路径:")
# for i, path in enumerate(sys.path):
#     print(f" {i}: {path}")

# # 验证cellcycle.py文件是否存在
# cellcycle_file = os.path.join(charmtools_path, "cellcycle.py")
# print(f"cellcycle.py文件是否存在: {os.path.exists(cellcycle_file)}")

# # 现在尝试导入模块
# try:
#     # 先尝试直接导入cellcycle模块
#     import cellcycle
#     print("✓ 成功导入cellcycle模块")
# except ImportError as e:
#     print(f"✗ 直接导入cellcycle失败: {e}")
#     try:
#         # 尝试从CHARMtools_snapshot导入
#         from CHARMtools import cellcycle
#         print("✓ 通过CHARMtools成功导入cellcycle")
#     except ImportError as e2:
#         print(f"✗ 通过CHARMtools_snapshot导入cellcycle失败: {e2}")

# # 尝试导入 CHARMio 模块
# try:
#     from CHARMtools import CHARMio
#     print("✓ 成功导入CHARMio模块")
# except ImportError as e:
#     print(f"✗ CHARMtools包不存在，跳过CHARMio导入: {e}")

# # 导入其他必需模块
# import glob
# import pandas as pd
# from concurrent import futures
# from functools import partial

# print("✓ 所有基础模块导入完成")


import sys
sys.path.append("/Volumes/SumSung500/CSU/0_HiRES/hires_code/HiRES/analysis_and_plot_notebooks")
# 现在尝试导入模块
try:
    # 先尝试直接导入cellcycle模块
    import cellcycle
    print("✓ 成功导入cellcycle模块")
except ImportError as e:
    print(f"✗ 直接导入cellcycle失败: {e}")
    try:
        # 尝试从CHARMtools_snapshot导入
        from CHARMtools import cellcycle
        print("✓ 通过CHARMtools成功导入cellcycle")
    except ImportError as e2:
        print(f"✗ 通过CHARMtools_snapshot导入cellcycle失败: {e2}")

# 尝试导入 CHARMio 模块
try:
    from CHARMtools import CHARMio
    print("✓ 成功导入CHARMio模块")
except ImportError as e:
    print(f"✗ CHARMtools包不存在，跳过CHARMio导入: {e}")


import glob
import pandas as pd

from concurrent import futures
from functools import partial

✗ 直接导入cellcycle失败: No module named 'cellcycle'
✓ 通过CHARMtools成功导入cellcycle
✓ 成功导入CHARMio模块


In [2]:
# 设置pairs文件路径，指向存储从GEO下载的pairs.gz文件的目录
PAIRS_PATH = "/Volumes/SumSung500/CSU/0_HiRES/data/hires/GSE223917_RAW/"

# 读取Excel文件，获取细胞元数据
metadata = pd.read_excel("/Volumes/SumSung500/CSU/0_HiRES/data/hires/GSE223917_HiRES_emb_metadata.xlsx")
print(f"Metadata中的细胞数量: {len(metadata)}")

# 只保留Cellname列，因为我们只需要细胞名称信息
cellnames = metadata[["Cellname"]].copy()

# 获取所有实际存在的pairs文件
import glob
all_pairs_files = glob.glob(f"{PAIRS_PATH}*.pairs.gz")
print(f"实际pairs文件数量: {len(all_pairs_files)}")

# 从文件名中提取GSM编号和细胞名的映射
gsm_cellname_mapping = {}
for file_path in all_pairs_files:
    filename = file_path.split('/')[-1]  # 获取文件名
    # 格式：GSM6998595_GasaE751001.pairs.gz
    if filename.startswith('GSM') and filename.endswith('.pairs.gz'):
        parts = filename.replace('.pairs.gz', '').split('_', 1)
        if len(parts) == 2:
            gsm_num = parts[0]
            cellname = parts[1]
            gsm_cellname_mapping[cellname] = gsm_num

print(f"成功匹配的GSM-细胞名映射数量: {len(gsm_cellname_mapping)}")

# 为metadata中的每个细胞查找对应的GSM编号和完整文件路径
def get_file_path(cellname):
    if cellname in gsm_cellname_mapping:
        gsm_num = gsm_cellname_mapping[cellname]
        return f"{PAIRS_PATH}{gsm_num}_{cellname}.pairs.gz"
    else:
        return None

cellnames["pairs"] = cellnames["Cellname"].apply(get_file_path)

# 检查是否有缺失的文件
missing_files = cellnames[cellnames["pairs"].isna()]
if len(missing_files) > 0:
    print(f"警告：有 {len(missing_files)} 个细胞的pairs文件不存在:")
    print(missing_files["Cellname"].tolist()[:10])  # 只显示前10个
    # 移除缺失的文件
    cellnames = cellnames.dropna()
    print(f"移除缺失文件后剩余细胞数量: {len(cellnames)}")

# 验证文件是否存在
import os
existing_files = cellnames["pairs"].apply(os.path.exists).sum()
print(f"验证存在的文件数量: {existing_files}/{len(cellnames)}")

# 显示前几行验证路径格式
print("\n生成的文件路径示例:")
print(cellnames.head())

# 将结果赋值给metadata变量供后续使用
metadata = cellnames

Metadata中的细胞数量: 7469
实际pairs文件数量: 7895
成功匹配的GSM-细胞名映射数量: 7895
验证存在的文件数量: 7469/7469

生成的文件路径示例:
      Cellname                                              pairs
0  GasaE751001  /Volumes/SumSung500/CSU/0_HiRES/data/hires/GSE...
1  GasaE751002  /Volumes/SumSung500/CSU/0_HiRES/data/hires/GSE...
2  GasaE751003  /Volumes/SumSung500/CSU/0_HiRES/data/hires/GSE...
3  GasaE751004  /Volumes/SumSung500/CSU/0_HiRES/data/hires/GSE...
4  GasaE751005  /Volumes/SumSung500/CSU/0_HiRES/data/hires/GSE...


In [3]:
# 这个单元格已被注释，因为上面的单元格已经正确设置了文件路径
# 为metadata中的每个细胞生成对应的pairs文件路径
# 使用lambda函数将细胞名称转换为完整的文件路径
# metadata["pairs"] = metadata["Cellname"].apply(lambda x: "../../../submitfiles/pairs_clean3/"+x+".pairs.gz")
# metadata["pairs"] = metadata["Cellname"].apply(lambda x: "/Volumes/SumSung500/CSU/0_HiRES/data/hires/GSE223917_RAW/"+x+".pairs.gz")

print("此单元格已被注释，使用上面单元格设置的正确文件路径")

此单元格已被注释，使用上面单元格设置的正确文件路径


In [4]:
# 计算细胞周期排序特征（ordering）
ordering = cellcycle.calc_cellcycle_ordering(metadata, threads=100)
# 计算接触描述特征（describe）
describe = cellcycle.calc_contact_describe(metadata, threads=100)
# 计算复制评分（repliscore），使用小鼠基因组的复制芯片数据
repliscore = cellcycle.calc_repli_score(
    metadata, 
    repli_chipf="/Volumes/SumSung500/CSU/0_HiRES/hires_code/HiRES/analysis_and_plot_notebooks/CHARMtools/ref/mm10_repli_chip.wig.gz", 
    threads=10
)

/Volumes/SumSung500/CSU/0_HiRES/hires_code/HiRES/analysis_and_plot_notebooks/CHARMtools/CHARMio.py:88: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  pairs = pd.read_table(filename, header=None, comment="#")
/Volumes/SumSung500/CSU/0_HiRES/hires_code/HiRES/analysis_and_plot_notebooks/CHARMtools/CHARMio.py:88: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  pairs = pd.read_table(filename, header=None, comment="#")
/Volumes/SumSung500/CSU/0_HiRES/hires_code/HiRES/analysis_and_plot_notebooks/CHARMtools/CHARMio.py:88: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  pairs = pd.read_table(filename, header=None, comment="#")
/Volumes/SumSung500/CSU/0_HiRES/hires_code/HiRES/analysis_and_plot_notebooks/CHARMtools/CHARMio.py:88: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.


In [5]:
# 将三种细胞周期特征（ordering、describe、repliscore）按列合并
res = pd.concat([ordering, describe, repliscore], axis=1)
# 可选：将结果保存为TSV文件（已注释）
res.to_csv("DNA_cellcycle.tsv", sep="\t", index=None)


In [6]:
# 显示合并后的结果DataFrame，查看细胞周期特征数据
res


,Cellname,pairs,order_index,near_p,mitotic_p,farAvg,NaganoCellcycle,repli_score,annote_ratio
0,GasaE751001,/Volumes/SumSung500/CSU/0_HiRES/data/hires/GSE...,6337,0.512112,0.202479,2.000648e+07,G1,0.585603,0.994522
1,GasaE751002,/Volumes/SumSung500/CSU/0_HiRES/data/hires/GSE...,1728,0.685396,0.119005,2.724219e+07,early/mid-S,0.618245,0.995242
2,GasaE751003,/Volumes/SumSung500/CSU/0_HiRES/data/hires/GSE...,1669,0.738354,0.121796,2.288031e+07,early/mid-S,0.548804,0.994558
3,GasaE751004,/Volumes/SumSung500/CSU/0_HiRES/data/hires/GSE...,7037,0.602727,0.341371,1.385462e+07,Pre-M,0.529733,0.993867
4,GasaE751005,/Volumes/SumSung500/CSU/0_HiRES/data/hires/GSE...,7081,0.405741,0.228987,1.879708e+07,G1,0.492611,0.992573
...,...,...,...,...,...,...,...,...,...
7464,OrgeEX053380,/Volumes/SumSung500/CSU/0_HiRES/data/hires/GSE...,1093,0.726202,0.122614,3.359666e+07,early/mid-S,0.577465,0.995228
7465,OrgeEX053381,/Volumes/SumSung500/CSU/0_HiRES/data/hires/GSE...,4878,0.576129,0.140135,3.913699e+07,G1,0.521149,0.993799
7466,OrgeEX053382,/Volumes/SumSung500/CSU/0_HiRES/data/hires/GSE...,628,0.733671,0.110580,3.218927e+07,early/mid-S,0.587758,0.994891
7467,OrgeEX053383,/Volumes/SumSung500/CSU/0_HiRES/data/hires/GSE...,6608,0.471231,0.182294,2.471220e+07,G1,0.524906,0.993949


In [7]:
# 使用ProcessPoolExecutor进行并发处理，计算每个细胞的dis_counts
# 使用100个进程并行处理，提高计算效率
with futures.ProcessPoolExecutor(10) as pool:
    res = pool.map(cellcycle.dis_counts, metadata["pairs"])
    
# 将结果转换为列表
ares = list(res)
# 创建DataFrame存储结果
cdps = pd.DataFrame(ares)
# 将列名转换为字符串类型
cdps.columns = cdps.columns.astype("string")
# 设置索引与metadata对应
cdps.index = metadata.reset_index().index


/Volumes/SumSung500/CSU/0_HiRES/hires_code/HiRES/analysis_and_plot_notebooks/CHARMtools/CHARMio.py:88: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  pairs = pd.read_table(filename, header=None, comment="#")
/Volumes/SumSung500/CSU/0_HiRES/hires_code/HiRES/analysis_and_plot_notebooks/CHARMtools/CHARMio.py:88: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  pairs = pd.read_table(filename, header=None, comment="#")
/Volumes/SumSung500/CSU/0_HiRES/hires_code/HiRES/analysis_and_plot_notebooks/CHARMtools/CHARMio.py:88: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  pairs = pd.read_table(filename, header=None, comment="#")
/Volumes/SumSung500/CSU/0_HiRES/hires_code/HiRES/analysis_and_plot_notebooks/CHARMtools/CHARMio.py:88: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.


In [1]:
import pandas as pd

# 读取metadata文件查看结构
metadata = pd.read_excel("/Volumes/SumSung500/CSU/0_HiRES/data/hires/GSE223917_HiRES_emb_metadata.xlsx")
print("Metadata文件的列名:")
print(metadata.columns.tolist())
print("\n前5行数据:")
print(metadata.head())

Metadata文件的列名:
['Cellname', 'Rawreads', 'DNAreads', 'RNAreads', 'y/x', 'Raw contacts', 'Dedup contacts', 'Clean1 contacts', 'Clean2 contacts', 'Clean3 contacts', 'Clean3 inter-contacts', 'Genes number', 'UMI counts', 'near%', 'mitotic%', 'farAvg', 'repli score', 'G1S.Score', 'G2M.Score', 'Stage', 'Celltype', 'RMSD 20k', 'CDPS cluster', 'Sub_k_cluster', 'Cellcycle phase']

前5行数据:
      Cellname  Rawreads  DNAreads  RNAreads       y/x  Raw contacts  \
0  GasaE751001  0.788963  0.704156  0.073214  0.010131        293142   
1  GasaE751002  1.194513  1.117651  0.065995  0.028088        593838   
2  GasaE751003  1.006697  0.938163  0.059082  0.021442        398891   
3  GasaE751004  1.127121  1.019122  0.094307  0.021927        407136   
4  GasaE751005  2.841285  2.650203  0.166877  0.023697       1095973   

   Dedup contacts  Clean1 contacts  Clean2 contacts  Clean3 contacts  ...  \
0          181153           180220           135168           135083  ...   
1          314892           313

In [10]:
import pandas as pd

# 读取metadata文件查看细胞数量
metadata = pd.read_excel("/Volumes/SumSung500/CSU/0_HiRES/data/hires/GSE223917_HiRES_emb_metadata.xlsx")
print(f"Metadata中的细胞数量: {len(metadata)}")
print(f"实际pairs.gz文件数量: 7895")

# 检查GSM编号范围
gsm_start = 6998595
gsm_end = gsm_start + len(metadata) - 1
print(f"计算出的GSM范围: GSM{gsm_start} - GSM{gsm_end}")

# 实际文件的GSM范围
print("实际文件范围: GSM6998595 - GSM7006716")

# 检查是否有间隔或重复
actual_gsm_start = 6998595
actual_gsm_end = 7006716
actual_count = actual_gsm_end - actual_gsm_start + 1
print(f"实际GSM连续数量: {actual_count}")

# 找到缺失的文件
if len(metadata) != 7895:
    print(f"警告：metadata文件中的细胞数量({len(metadata)})与实际文件数量(7895)不匹配！")

Metadata中的细胞数量: 7469
实际pairs.gz文件数量: 7895
计算出的GSM范围: GSM6998595 - GSM7006063
实际文件范围: GSM6998595 - GSM7006716
实际GSM连续数量: 8122
警告：metadata文件中的细胞数量(7469)与实际文件数量(7895)不匹配！


In [2]:
cdps

NameError: name 'cdps' is not defined